In [2]:
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd


In [ ]:
df = pd.read_csv('datasets/OnlineRetail.csv',encoding='ISO-8859-1')
df = df.dropna()
#drop the rows that have negative values in Quantity column
df = df[df['Quantity']>0]
#drop the rows that have negative values in UnitPrice column
df = df[df['UnitPrice']>0]


In [4]:
df['Amount'] = df['Quantity'] * df['UnitPrice']
monetary = df.groupby('CustomerID')['Amount'].sum()
monetary = monetary.reset_index()
monetary.head()

,CustomerID,Amount
0,12346.0,77183.60
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40


In [5]:
frequency = df.groupby('CustomerID')['InvoiceNo'].count()
frequency = frequency.reset_index()
frequency.head()

,CustomerID,InvoiceNo
0,12346.0,1
1,12347.0,182
2,12348.0,31
3,12349.0,73
4,12350.0,17


In [6]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'],format='%d-%m-%Y %H:%M')
df['diff'] = max(df['InvoiceDate']) - df['InvoiceDate']
recency = df.groupby('CustomerID')['diff'].min()
recency = recency.reset_index()
recency.head()  

,CustomerID,diff
0,12346.0,325 days 02:49:00
1,12347.0,1 days 20:58:00
2,12348.0,74 days 23:37:00
3,12349.0,18 days 02:59:00
4,12350.0,309 days 20:49:00


In [7]:
rfm = pd.merge(recency, frequency, on='CustomerID', how='inner')
rfm = pd.merge(rfm, monetary, on='CustomerID', how='inner')
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
#turn customerID into integer
rfm['CustomerID'] = rfm['CustomerID'].astype(int)
#turn recency into integer(days)
rfm['Recency'] = rfm['Recency'].dt.days

#normalize the data
rfm['Recency'] = (rfm['Recency'] - rfm['Recency'].mean())/rfm['Recency'].std()
rfm['Frequency'] = (rfm['Frequency'] - rfm['Frequency'].mean())/rfm['Frequency'].std()
rfm['Monetary'] = (rfm['Monetary'] - rfm['Monetary'].mean())/rfm['Monetary'].std()
rfm.head()


,CustomerID,Recency,Frequency,Monetary
0,12346,2.334305,-0.396532,8.357705
1,12347,-0.905236,0.394603,0.250937
2,12348,-0.175339,-0.265405,-0.028593
3,12349,-0.735260,-0.081826,-0.033008
4,12350,2.174328,-0.326597,-0.191325


In [8]:
rfm.to_csv('rfm.csv',index=False)